In [1]:
%cd /opt/homebrew/Cellar/openslide/3.4.1_5/lib
!pwd


/opt/homebrew/Cellar/openslide/3.4.1_5/lib
/opt/homebrew/Cellar/openslide/3.4.1_5/lib


In [1]:
from models.model_clam import CLAM_SB
from models.model_clam import CLAM_MB
import torch
ckpt_path = "/Users/M261759/Documents/GitHub/CLAM/trained_params/tcga_cptac_lung_cv/lung_public_cv_CLAM_50_s1/s_8_checkpoint.pt"
model_dict = {"dropout": False, 'n_classes': 2}


model_size = 'small'
model_type = 'clam_sb'

if model_size is not None and model_type in ['clam_sb', 'clam_mb']:
    model_dict.update({"size_arg": model_size})

model = CLAM_MB(**model_dict)


ckpt = torch.load(ckpt_path, map_location=torch.device('cpu'))
ckpt_clean = {}
for key in ckpt.keys():
    if 'instance_loss_fn' in key:
        continue
    new_key = key.replace('.module', '')
    new_key = new_key.replace('attention_net.3', 'attention_net.2')
    ckpt_clean.update({new_key: ckpt[key]})

#for key, value in ckpt_clean.items() :
    #print (key)
model.load_state_dict(ckpt_clean, strict=True)

sample_input = torch.rand(size = (100, 1024))

logits, Y_prob, Y_hat, A_raw, results_dict = model(sample_input)
print(logits)
print(A_raw.shape)
#print(model)

slide_path = "/Users/M261759/Documents/GitHub/CLAM/slides/LUSC/TCGA-60-2726-11A-01-TS1.cae5172e-ceca-4939-bc3f-e48657e371fb.svs"

tensor([[ 21.5446, -37.8466]], grad_fn=<CopySlices>)
torch.Size([2, 100])


In [36]:
#generate alternative tiles for a list of tiles
def adversarial_attack_wsi(model_CLAM, feature_extractor, true_label, tiles):
    pass

In [82]:
import openslide
import numpy as np
from tqdm import tqdm
import cv2

def isWhitePatch(patch, satThresh=5):
    patch_hsv = cv2.cvtColor(patch, cv2.COLOR_RGB2HSV)
    return True if np.mean(patch_hsv[:,:,1]) < satThresh else False

def isBlackPatch(patch, rgbThresh=40):
    return True if np.all(np.mean(patch, axis = (0,1)) < rgbThresh) else False

def isBlackPatch_S(patch, rgbThresh=20, percentage=0.05):
    num_pixels = patch.size
    return True if np.all(np.array(patch) < rgbThresh, axis=(2)).sum() > num_pixels * percentage else False

def isWhitePatch_S(patch, rgbThresh=220, percentage=0.2):
    num_pixels = patch.size
    return True if np.all(np.array(patch) > rgbThresh, axis=(2)).sum() > num_pixels * percentage else False

def small_feature_extraction(wholeslide_path, feature_extractor, tile_dims = [224,224], check_filter = True):
    slide = openslide.OpenSlide(wholeslide_path)
    index_xy = {}
    slide_x, slide_y = slide.dimensions
    print("Whole Slide Dims: {}".format(slide.dimensions))
    tile_dim_x, tile_dim_y = tile_dims
    num_tiles_x, num_tiles_y =  (int)(slide_x/tile_dim_x), (int)(slide_y/tile_dim_y)
    total_tiles = (int)(slide_x/tile_dim_x)*(int)(slide_y/tile_dim_y)
    print("Number of tiles: {}".format(total_tiles))
    output = np.zeros(shape=(num_tiles_x, num_tiles_y, 1024))
    index = 0
    for i_x in tqdm(range(num_tiles_x)):
        for i_y in range(num_tiles_y):
            curr_x = i_x*tile_dim_x
            curr_y = i_y*tile_dim_y
            index_xy.update({index:(curr_x, curr_y)})
            index += 1
            tile_curr = slide.read_region(location=(curr_x, curr_y), level=1, size = (tile_dim_x, tile_dim_y))
            tile_curr = np.array(tile_curr)[:,:,0:3]
            if check_filter:
                if isBlackPatch_S(tile_curr, rgbThresh=20, percentage=0.05) or isWhitePatch_S(tile_curr, rgbThresh=220, percentage=0.25):
                    continue

            tile_curr = torch.unsqueeze(torch.tensor(np.transpose(np.array(tile_curr),(2,0,1))),dim=0)/255
            features = feature_extractor(tile_curr)
            output[i_x, i_y, :] = features.detach().numpy()

    return output, index_xy

In [83]:
import torch
from models.resnet_custom import ResNet_Baseline, resnet50_baseline

feature_extractor = resnet50_baseline(pretrained=True)
features, index_xy = small_feature_extraction(slide_path, feature_extractor, )

Whole Slide Dims: (16000, 19023)
Number of tiles: 5964


100%|██████████| 71/71 [01:54<00:00,  1.62s/it]


In [84]:
#features.shape
print(index_xy)
features = torch.flatten(torch.tensor(features), start_dim=0, end_dim=1)
features.shape

{0: (0, 0), 1: (0, 224), 2: (0, 448), 3: (0, 672), 4: (0, 896), 5: (0, 1120), 6: (0, 1344), 7: (0, 1568), 8: (0, 1792), 9: (0, 2016), 10: (0, 2240), 11: (0, 2464), 12: (0, 2688), 13: (0, 2912), 14: (0, 3136), 15: (0, 3360), 16: (0, 3584), 17: (0, 3808), 18: (0, 4032), 19: (0, 4256), 20: (0, 4480), 21: (0, 4704), 22: (0, 4928), 23: (0, 5152), 24: (0, 5376), 25: (0, 5600), 26: (0, 5824), 27: (0, 6048), 28: (0, 6272), 29: (0, 6496), 30: (0, 6720), 31: (0, 6944), 32: (0, 7168), 33: (0, 7392), 34: (0, 7616), 35: (0, 7840), 36: (0, 8064), 37: (0, 8288), 38: (0, 8512), 39: (0, 8736), 40: (0, 8960), 41: (0, 9184), 42: (0, 9408), 43: (0, 9632), 44: (0, 9856), 45: (0, 10080), 46: (0, 10304), 47: (0, 10528), 48: (0, 10752), 49: (0, 10976), 50: (0, 11200), 51: (0, 11424), 52: (0, 11648), 53: (0, 11872), 54: (0, 12096), 55: (0, 12320), 56: (0, 12544), 57: (0, 12768), 58: (0, 12992), 59: (0, 13216), 60: (0, 13440), 61: (0, 13664), 62: (0, 13888), 63: (0, 14112), 64: (0, 14336), 65: (0, 14560), 66: (

torch.Size([5964, 1024])

In [85]:
features = features.float()
logits, Y_prob, Y_hat, A_raw, results_dict = model(features)

In [86]:
print(Y_prob)

tensor([[0.4927, 0.5073]], grad_fn=<SoftmaxBackward0>)


In [87]:
import torch.nn.functional as F
from torch.autograd import grad

def generate_attack_tiles(slide_path, model, feature_extractor, index_xy, pre_calculated_features, target_label, tile_target = None, eps = 300, tile_size = [224,224]):
    '''
    tile target: iterable of tuples in format(tile#_x, tile#_y)
    index_xy a dictionary index -> xy spots on the slide
    precalculated feature: tiles_x, tiles_y, features(1024)
    '''
    slide = openslide.OpenSlide(slide_path)
    slide_x, slide_y = slide.dimensions
    tile_dim_x, tile_dim_y = tile_size
    default_top_k = 20
    num_tiles_x, num_tiles_y =  (int)(slide_x/tile_dim_x), (int)(slide_y/tile_dim_y)
    copy_features = torch.tensor(pre_calculated_features.clone(), requires_grad=True)
    print(copy_features.shape)
    logits, Y_prob, Y_hat, A_raw, results_dict = model(copy_features)
    loss = F.nll_loss(logits, torch.unsqueeze(target_label,dim=0))
    loss.backward()
    print(copy_features.grad)
    if tile_target == None:
        #default top k
        print("Models says {}, true answer is {}".format(logits, target_label))
        ind = torch.topk(A_raw[target_label,:], k = default_top_k).indices
        tile_target = ind.detach().numpy()

    num_attack_tiles = len(tile_target)
    attack_tiles = []
    clone_grad = copy_features.grad.clone()
    for i, index in enumerate(tile_target):
        curr_x, curr_y = index_xy[index]
        tile_curr = slide.read_region(location=(curr_x, curr_y), level=0, size = (tile_dim_x, tile_dim_y))
        tile_curr = torch.unsqueeze(torch.tensor(np.transpose(np.array(tile_curr,dtype=float)[:,:,0:3],(2,0,1)), requires_grad=True), dim = 0)
        features_curr = feature_extractor(tile_curr.float())
        target_grad = clone_grad[index: index+1,:]
        psudo_label = (target_grad/2 + features_curr).detach()
        curr_loss = torch.sum((features_curr - psudo_label)**2)
        d_tc_y = grad(outputs=curr_loss, inputs=tile_curr)[0]
        feature_extractor.zero_grad()
        print("dty_c.shape {}".format(d_tc_y[0].shape))
        tile_curr_adv = tile_curr + eps * d_tc_y.sign()
        attack_tiles.append(tile_curr_adv)


    return attack_tiles, tile_target

In [88]:
attack_tiles,tile_target = generate_attack_tiles(slide_path = slide_path, model = model, feature_extractor = feature_extractor, index_xy = index_xy, pre_calculated_features = features, target_label = torch.tensor(0), eps = 10)

/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/922121340.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  copy_features = torch.tensor(pre_calculated_features.clone(), requires_grad=True)


torch.Size([5964, 1024])
tensor([[-2.9957e-05,  1.3888e-04, -1.4162e-04,  ...,  5.6554e-05,
         -4.3817e-05, -2.4358e-05],
        [-2.9957e-05,  1.3888e-04, -1.4162e-04,  ...,  5.6554e-05,
         -4.3817e-05, -2.4358e-05],
        [-2.9957e-05,  1.3888e-04, -1.4162e-04,  ...,  5.6554e-05,
         -4.3817e-05, -2.4358e-05],
        ...,
        [-2.9957e-05,  1.3888e-04, -1.4162e-04,  ...,  5.6554e-05,
         -4.3817e-05, -2.4358e-05],
        [-2.9957e-05,  1.3888e-04, -1.4162e-04,  ...,  5.6554e-05,
         -4.3817e-05, -2.4358e-05],
        [-2.9957e-05,  1.3888e-04, -1.4162e-04,  ...,  5.6554e-05,
         -4.3817e-05, -2.4358e-05]])
Models says tensor([[-0.0227,  0.0064]], grad_fn=<CopySlices>), true answer is 0
dty_c.shape torch.Size([3, 224, 224])
dty_c.shape torch.Size([3, 224, 224])
dty_c.shape torch.Size([3, 224, 224])
dty_c.shape torch.Size([3, 224, 224])
dty_c.shape torch.Size([3, 224, 224])
dty_c.shape torch.Size([3, 224, 224])
dty_c.shape torch.Size([3, 224, 22

In [89]:
def top_k_tile_attack(slide_path, model, feature_extractor, pre_calculated_features, index_xy, attack_tiles, tile_target,):
    #generate new features
    new_features = pre_calculated_features.clone()
    for i, index in enumerate(tile_target):
        curr_x, curr_y = index_xy[index]
        new_feature_i = feature_extractor(attack_tiles[i].float())
        new_features[index, :] = new_feature_i
    logits, Y_prob, Y_hat, A_raw, results_dict = model(new_features)
    return Y_prob, A_raw

In [90]:
features.shape
y,a = top_k_tile_attack(slide_path, model=model, feature_extractor=feature_extractor, pre_calculated_features=features, index_xy=index_xy, attack_tiles=attack_tiles, tile_target=tile_target)
print(y)
print(F.softmax(a))
print(F.softmax(A_raw))
print(torch.sum(torch.abs(F.softmax(a)-F.softmax(A_raw))))
print(torch.max(F.softmax(A_raw)))

tensor([[0.4927, 0.5073]], grad_fn=<SoftmaxBackward0>)
tensor([[1.0505e-07, 1.3928e-07, 1.4692e-07,  ..., 3.1251e-04, 3.1251e-04,
         3.1251e-04],
        [9.2183e-08, 1.2637e-07, 1.3301e-07,  ..., 3.1252e-04, 3.1252e-04,
         3.1252e-04]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003]],
       grad_fn=<SoftmaxBackward0>)
tensor(0.0248, grad_fn=<SumBackward0>)
tensor(0.0003, grad_fn=<MaxBackward1>)


/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/1839246975.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(a))
/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/1839246975.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(A_raw))
/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/1839246975.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.sum(torch.abs(F.softmax(a)-F.softmax(A_raw))))
/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/1839246975.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.max(F.softmax(A_raw)))


In [91]:
a.shape#%%
a_interested = a[:,tile_target]
a_raw_interested = A_raw[:, tile_target]

print(a_interested)
print(a_raw_interested)

tensor([[-7.8618, -7.5733, -7.7706, -7.8246, -8.2394, -8.0043, -8.3602, -7.9367,
         -8.1136, -8.0867, -7.8370, -8.2547, -7.8189, -8.1822, -8.1343, -8.2722,
         -7.9901, -8.3778, -8.0437, -8.2329],
        [-7.9240, -7.6507, -7.8274, -7.8967, -8.3254, -8.0668, -8.4457, -7.9987,
         -8.2020, -8.1701, -7.9101, -8.3234, -7.8803, -8.2650, -8.2081, -8.3653,
         -8.0499, -8.4662, -8.1053, -8.3129]], grad_fn=<IndexBackward0>)
tensor([[-0.2743, -0.2743, -0.2743, -0.2743, -0.2743, -0.2743, -0.2743, -0.2743,
         -0.2743, -0.2743, -0.2743, -0.2743, -0.2743, -0.2743, -0.2743, -0.2743,
         -0.2743, -0.2743, -0.2743, -0.2743],
        [-0.2367, -0.2367, -0.2367, -0.2367, -0.2367, -0.2367, -0.2367, -0.2367,
         -0.2367, -0.2367, -0.2367, -0.2367, -0.2367, -0.2367, -0.2367, -0.2367,
         -0.2367, -0.2367, -0.2367, -0.2367]], grad_fn=<IndexBackward0>)


In [92]:
torch.sum(torch.abs(a_interested-a_raw_interested))

tensor(313.0903, grad_fn=<SumBackward0>)

In [93]:
print(torch.mean(A_raw))

tensor(-3.9998, grad_fn=<MeanBackward0>)


In [100]:
import matplotlib
def feature_attack(model, precalculated_features, eps = .2, target = 0):
    copy_feature = precalculated_features.clone()
    copy_feature = torch.tensor(copy_feature, requires_grad=True)
    logits, Y_prob, Y_hat, A_raw_natural, results_dict = model(copy_feature)

    print(logits)

    ind = torch.topk(A_raw[target,:], k = 100).indices
    tile_target = ind

    target = torch.Tensor([target]).long()
    loss = F.nll_loss(logits, target)
    loss.backward()
    d_x_y = copy_feature.grad
    #torch.sign(d_x_y)[tile_target,:] = 0
    attack = copy_feature + d_x_y*eps

    logits, Y_prob, Y_hat, A_raw_after, results_dict = model(attack)
    print(logits)
    print(F.softmax(A_raw_natural))
    print(F.softmax(A_raw_after))




In [101]:
feature_attack(model, features, target=1)


tensor([[-0.0227,  0.0064]], grad_fn=<CopySlices>)
tensor([[-0.0181,  0.0015]], grad_fn=<CopySlices>)
tensor([[0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003]],
       grad_fn=<SoftmaxBackward0>)
tensor([[0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003],
        [0.0003, 0.0003, 0.0003,  ..., 0.0003, 0.0003, 0.0003]],
       grad_fn=<SoftmaxBackward0>)


/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/3881024821.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  copy_feature = torch.tensor(copy_feature, requires_grad=True)
/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/3881024821.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(A_raw_natural))
/var/folders/5k/67vrc2h904n6ls3t_k7mxgcr0000gs/T/ipykernel_84869/3881024821.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(A_raw_after))
